In [ ]:
%pip install -qU langchain-community tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.6 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import RetrievalQA

# 🔹 OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
os.environ["OPENAI_API_KEY"] = "openai-api-key"


In [ ]:
!wget https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/voc_example.txt

--2025-03-24 10:25:17--  https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/voc_example.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2306 (2.3K) [text/plain]
Saving to: ‘voc_example.txt.1’

voc_example.txt.1   100%[===================>]   2.25K  --.-KB/s    in 0s      

2025-03-24 10:25:17 (27.6 MB/s) - ‘voc_example.txt.1’ saved [2306/2306]



In [ ]:
# 1. 데이터 로딩
loader = TextLoader('voc_example.txt')
documents = loader.load()

In [ ]:
# 2. 텍스트 분할 (Chunking)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, separators=["\n\n"])
docs = text_splitter.split_documents(documents)

docs

[Document(metadata={'source': 'voc_example.txt'}, page_content='[고객1]\n배송이 너무 느립니다. 일주일이나 걸렸어요. 이런 식이면 다음부터는 다른 쇼핑몰을 이용해야 할 것 같네요.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객2]\n제품 포장은 잘 되어 있었지만, 설명서가 너무 불친절합니다. 사용법이 이해가 안 가서 검색해봤어요. 초보자 입장에선 너무 어렵습니다.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객3]\n배송은 빠르게 왔는데, 박스가 찌그러져 있었습니다. 선물용이라 포장이 중요했는데 좀 아쉽네요.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객4]\n제품은 괜찮은데 고객센터 연결이 너무 어렵습니다. 10번 넘게 전화해서 겨우 연결했어요. 상담원은 친절했지만 대기 시간이 너무 길었습니다.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객5]\n설명서에 나와 있는 기능이 실제 제품에는 없네요. 사기당한 기분입니다. 반품 요청하고 싶은데 절차가 너무 복잡합니다.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객6]\n택배가 도착했는데 문 앞에 아무런 연락 없이 그냥 두고 갔어요. 분실 우려가 있어서 불안했습니다.'),
 Document(metadata={'source': 'voc_example.txt'}, page_content='[고객7]\n주문한 날짜 기준으로 정확히 7일 만에 도착했어요. 요즘 시대에 너무 느린 거 아닌가요?'),
 Document(metadata={'source': 'voc_example.tx

In [ ]:
# 3. 임베딩 생성 및 벡터DB 저장
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

# 4. 기본 Retriever 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
# 5️. Contextual Compression 적용
llm = ChatOpenAI(model_name="gpt-4o-mini")  # 압축에 사용할 LLM
compressor = LLMChainExtractor.from_llm(llm)  # LLM 기반 문서 압축기
compressed_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,  # 이 부분이 수정됨 (compressor -> base_compressor)
    base_retriever=retriever,
)

# 6️. RAG QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=compressed_retriever)

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
# 7️. 질문 입력 후 응답 확인
query = "고객 VOC에서, 배송과 관련된 문제를 언급한 고객 피드백에 대해 정리해주세요."
response = qa_chain.run(query)

print("💡 답변:", response)

# retrieval 결과 비교
print("\n\n##### normal retriever #####")
pretty_print_docs(retriever.invoke(query))

print("\n\n##### compressed retriever #####")
pretty_print_docs(compressed_retriever.invoke(query))

💡 답변: 고객 VOC에서 배송과 관련된 문제를 언급한 고객 피드백은 다음과 같습니다:

1. **배송 준비 상태**: 고객14는 주문 후 배송 준비 상태에서 며칠간 대기한 뒤 갑자기 발송되었다고 언급하며, 중간 진행 상황을 더 잘 공유해주길 요청했습니다.

2. **배송 속도**: 고객1은 배송이 너무 느리다고 불만을 표하며, 일주일이 걸렸다고 언급했습니다. 이러한 상황이 계속된다면 다른 쇼핑몰을 이용할 것이라고 경고했습니다.

3. **배송 박스 크기**: 고객10은 배송 박스가 너무 커서 불편하다고 언급하며, 작은 물건 하나를 보내는데 큰 상자를 사용하는 것에 대해 의문을 제기했습니다.


##### normal retriever #####
Document 1:

[고객14]
주문 후 배송 준비 상태로 며칠이나 있다가 갑자기 발송되더라고요. 중간 진행 상황이 좀 더 잘 공유되면 좋겠습니다.
----------------------------------------------------------------------------------------------------
Document 2:

[고객1]
배송이 너무 느립니다. 일주일이나 걸렸어요. 이런 식이면 다음부터는 다른 쇼핑몰을 이용해야 할 것 같네요.
----------------------------------------------------------------------------------------------------
Document 3:

[고객11]
제품은 만족스러운데 설명서에 오타가 많아서 신뢰가 떨어졌습니다. 검수 좀 철저히 해주세요.
----------------------------------------------------------------------------------------------------
Document 4:

[고객4]
제품은 괜찮은데 고객센터 연결이 너무 어렵습니다. 10번 넘게 전화해서 겨우 연결했어요. 상담원은 친절했지만 대기 시간이 너무 길었습니다.

In [ ]:
# 7️. 질문 입력 후 응답 확인
query = "상품 설명과 관련된 이슈가 있는 피드백만 요약하려고 합니다. 어떤 피드백이 해당되며, 요약된 핵심 내용은 무엇인가요?"
response = qa_chain.run(query)

print("💡 답변:", response)

# retrieval 결과 비교
print("\n\n##### normal retriever #####")
pretty_print_docs(retriever.invoke(query))

print("\n\n##### compressed retriever #####")
pretty_print_docs(compressed_retriever.invoke(query))

💡 답변: 상품 설명과 관련된 이슈가 있는 피드백은 다음과 같습니다:

1. **고객9**:
   - 제품 설명서가 너무 간단하여 기능을 제대로 알 수 없음.
   - 설명서에 오타가 많아 신뢰가 떨어짐. 검수가 필요함.

2. **고객8**:
   - 설명서가 외국어로만 되어 있어 이해하기 어려움. 한글 설명서 요청.
   - 설명서에 나와 있는 기능이 실제 제품에는 없음. 반품 요청 절차가 복잡함.
   - 설명서가 불친절하고 사용법이 어렵다고 느낌.

**핵심 내용 요약**:
- 제품 설명서의 간결함과 오타 문제로 인해 고객들이 기능을 이해하기 어렵고 신뢰를 잃고 있음.
- 외국어 설명서로 인해 이해의 어려움이 있으며, 한글 설명서가 필요함.
- 설명서와 실제 제품 간의 불일치로 인해 반품 요청이 복잡하다고 느끼고 있음.


##### normal retriever #####
Document 1:

[고객9]
제품 설명서가 너무 간단해서 어떤 기능이 있는지 제대로 알 수 없었습니다. 상세한 설명이 필요합니다.
----------------------------------------------------------------------------------------------------
Document 2:

[고객11]
제품은 만족스러운데 설명서에 오타가 많아서 신뢰가 떨어졌습니다. 검수 좀 철저히 해주세요.
----------------------------------------------------------------------------------------------------
Document 3:

[고객8]
설명서가 외국어로만 되어 있어서 내용을 이해하기 어려웠습니다. 한글 설명서도 같이 제공해 주세요.
----------------------------------------------------------------------------------------------------
Document 4:

[고객5]
설명서에 나와 있는 기능이

## 실습과제

- 주어진 논문 텍스트에서, 정책 제안과 관련된 내용만 추출해 핵심적으로 요약해보세요.
(힌트: 결론 부분을 중심으로 확인)

- "미세먼지와 건강의 관계"에 대한 핵심 연구 결과만 2~3줄로 압축하여 요약하세요.
(불필요한 연구 설계나 데이터 출처는 제외)

In [ ]:
!wget https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/paper_example.txt

--2025-03-24 10:29:59--  https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/paper_example.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1603 (1.6K) [text/plain]
Saving to: ‘paper_example.txt’

paper_example.txt   100%[===================>]   1.57K  --.-KB/s    in 0s      

2025-03-24 10:29:59 (22.1 MB/s) - ‘paper_example.txt’ saved [1603/1603]



In [ ]:
# 1. 데이터 로딩
loader = TextLoader('paper_example.txt')
documents = loader.load()

In [ ]:
# 2. 텍스트 분할 (Chunking)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, separators=["\n\n"])
docs = text_splitter.split_documents(documents)

docs

[Document(metadata={'source': 'paper_example.txt'}, page_content='\n[초록(Abstract)]\n본 연구는 미세먼지 농도가 도시민의 건강에 미치는 영향을 실증적으로 분석하였다. 2018년부터 2022년까지의 대기질 자료와 병원 진료 데이터를 바탕으로 패널 회귀모형을 구축하였으며, PM2.5의 농도가 높을수록 호흡기 질환 발생률이 유의하게 증가하는 것으로 나타났다.'),
 Document(metadata={'source': 'paper_example.txt'}, page_content='\n\n[서론(Introduction)]\n최근 몇 년간 미세먼지에 대한 관심이 급증하고 있으며, 정부에서도 다양한 대책을 발표하고 있다. 그러나 실제로 미세먼지가 건강에 어떤 영향을 미치는지는 아직 논란이 많다. 이에 본 연구는 실증적 데이터를 활용하여 미세먼지와 건강 간의 상관관계를 규명하고자 한다.'),
 Document(metadata={'source': 'paper_example.txt'}, page_content='\n\n[방법(Methodology)]\n본 연구에서는 환경부에서 제공하는 전국 17개 시도의 대기질 측정 데이터를 활용하였다. 또한 건강보험심사평가원에서 제공하는 병원 진료 기록을 기반으로 질병 발생률을 집계하였다. 분석에는 고정효과 모형과 다중회귀모형을 적용하였다.'),
 Document(metadata={'source': 'paper_example.txt'}, page_content='\n\n[결과(Results)]\nPM2.5의 평균 농도가 10㎍/㎥ 증가할 때마다 호흡기 질환 진료율은 약 5.2% 증가하였다. 특히, 어린이와 노인 집단에서 그 효과가 더 뚜렷하게 나타났다.'),
 Document(metadata={'source': 'paper_example.txt'}, page_content='\n\n[결론(Conclusion)]\n미세먼지는 호흡기 건강에 유의미한 영향을 미치며, 특히 취약 계층

In [ ]:
# 3. 임베딩 생성 및 벡터DB 저장
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

# 4. 기본 Retriever 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
# 5️. Contextual Compression 적용
llm = ChatOpenAI(model_name="gpt-4o-mini")  # 압축에 사용할 LLM
compressor = LLMChainExtractor.from_llm(llm)  # LLM 기반 문서 압축기
compressed_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,  # 이 부분이 수정됨 (compressor -> base_compressor)
    base_retriever=retriever,
)

# 6️. RAG QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=compressed_retriever)

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
# 7️. 질문 입력 후 응답 확인
query = "아래 논문 텍스트에서, 정책 제안과 관련된 내용만 추출해 핵심적으로 요약해보세요."
response = qa_chain.run(query)

print("💡 답변:", response)

print("##### normal retriever #####")
pretty_print_docs(retriever.invoke(query))

print("##### compressed retriever #####")
pretty_print_docs(compressed_retriever.invoke(query))

💡 답변: 논문에서는 정부가 미세먼지로 인한 호흡기 건강 영향을 최소화하기 위해 실효성 있는 정책을 수립해야 하며, 취약 계층을 보호하기 위한 지속적인 모니터링 체계를 마련할 필요성이 강조되고 있습니다.
[normal retriever]
Document 1:



[결론(Conclusion)]
미세먼지는 호흡기 건강에 유의미한 영향을 미치며, 특히 취약 계층에 대한 보호가 필요함을 시사한다. 정부는 실효성 있는 정책 수립과 함께 지속적인 모니터링 체계를 마련해야 할 것이다.

----------------------------------------------------------------------------------------------------
Document 2:



[서론(Introduction)]
최근 몇 년간 미세먼지에 대한 관심이 급증하고 있으며, 정부에서도 다양한 대책을 발표하고 있다. 그러나 실제로 미세먼지가 건강에 어떤 영향을 미치는지는 아직 논란이 많다. 이에 본 연구는 실증적 데이터를 활용하여 미세먼지와 건강 간의 상관관계를 규명하고자 한다.
----------------------------------------------------------------------------------------------------
Document 3:



[방법(Methodology)]
본 연구에서는 환경부에서 제공하는 전국 17개 시도의 대기질 측정 데이터를 활용하였다. 또한 건강보험심사평가원에서 제공하는 병원 진료 기록을 기반으로 질병 발생률을 집계하였다. 분석에는 고정효과 모형과 다중회귀모형을 적용하였다.
----------------------------------------------------------------------------------------------------
Document 4:


[초록(Abstract)]
본 연구는 미세먼지 농도가 도시민의 건강에 미치는 영향을 실증적으로 분석하였다. 2018년부터 2022